In [3]:
import os
os.getcwd()
os.chdir("/Users/Suncicie/Study/DataMining/Competition/Vips")
import numpy as np
import pandas as  pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from scipy import sparse
import gc
import pickle

In [5]:
# user_action_train=pd.read_table("data/user_action_train.txt",header=None)
# goods=pd.read_table("data/goods_train.txt",header=None)
# user_action_test=pd.read_table("data/user_action_test_items.txt",header=None)
# 
# user_action_train.columns=["user_id","spu_id","bought","date"]
# user_action_test.columns=["user_id","spu_id","prob"]
# goods.columns=["spu_id","brand_id","cat_id"]
# 
# # first the lr
# train=pd.merge(user_action_train,goods,on="spu_id",how="left")
# test=pd.merge(user_action_test,goods,on="spu_id",how="left")

print train.head()
print test.head()

dump_path_train="data/origin_train.pkl"
dump_path_test="data/origin_test.pkl"
pickle.dump(train, open(dump_path_train, 'w'))
pickle.dump(test, open(dump_path_test, 'w'))

In [8]:
# ---construct the sum of a user bought in the 3 months
# ---instslled_count = dfInstalled[['userID','count']].groupby('userID').sum().reset_index()

UserClickDF=train[["user_id","bought"]].groupby("user_id").count().reset_index()
UserClickDF.columns=["user_id","click_count"]
print UserClickDF.head()
# # # # # 
UserBoughtDF=train[["user_id","bought"]].groupby("user_id").sum().reset_index()
UserBoughtDF.columns=["user_id","bought_count"]
print UserBoughtDF.head()
# # 

# print train.columns
# print test.columns
train=pd.merge(train,UserClickDF,on="user_id",how="left")
test=pd.merge(test,UserClickDF,on="user_id",how="left")
# # # # 
# print train.columns
# print test.columns
train=pd.merge(train,UserBoughtDF,on="user_id",how="left")
test=pd.merge(test,UserBoughtDF,on="user_id",how="left")
# # # 
# # # 
# print train.columns
# print test.columns
# 

train["bought_rate"]=np.rint(((train["bought_count"]/train["click_count"])*100000000)/100000)
test["bought_rate"]=np.rint(((test["bought_count"]/test["click_count"])*100000000)/100000)
print train.head()
print test.head()

# construct the sum of a user click in the 3 months
# construct the sum of a user bought in the 3 months
# construct the rate of bought/click in the 3 months
# construct the mean bought of a month #this has no sense for it have sum as a feature

   user_id  spu_id  bought   date    brand_id  cat_id  click_count  \
0   320510  839213       0  03-10  10010631.0   271.0          141   
1   314210  369282       0  01-05  10001351.0   311.0          489   
2   381441  730484       0  01-05  10000866.0   311.0           39   
3   168858  902515       0  03-10  10028088.0   297.0          170   
4   270694  953270       0  01-30  10000223.0  1056.0          335   

   bought_count  bought_rate  
0             0          0.0  
1             2          4.0  
2             0          0.0  
3             0          0.0  
4             4         12.0  
   user_id   spu_id  prob  brand_id  cat_id  click_count  bought_count  \
0    60750  1580520   NaN  10012892   28008          203             4   
1   595361   484220   NaN  10005367     311          296             0   
2    45427   326736   NaN  10004119   28006         1248            21   
3   443345  1049603   NaN  10012721     680          379             2   
4   318510    74820   N

In [9]:
# encoding="utf-8"
# ---the feature about brand, it show whether a user like buy one brand
# the number of a user cat brand in 3 month 
UserClickBrand=train.groupby(["user_id"])["brand_id"].value_counts()
UserClickBrand=UserClickBrand.rename("brand_id_count").reset_index()
print UserClickBrand.head()
# #  user_id    brand_id  brand_id_count
# # 0        3  10004318.0              19
# # 1        3  10005367.0              18
# # 2        3  10013106.0              17
# # 3        3  10020991.0              17
# # 4        3  10000601.0              14
# # 
UserClickBrandCount=UserClickBrand.groupby(["user_id"])["brand_id"].count().reset_index()
UserClickBrandCount.columns=["user_id","brand_count"]
print UserClickBrandCount.head()
#   # user_id  brand_id
# # 0        3       108
# # 1        4        60
# # 2       11        21
# # 3       16        17
# # 4       17       407
train=pd.merge(train,UserClickBrandCount,on="user_id",how="left")
test=pd.merge(test,UserClickBrandCount,on="user_id",how="left")
# # 
# # # ---the number of a user cat goods in 3 month
# # # print UserClickBrand2.head()
UserClickGoodsCount=UserClickBrand.groupby(["user_id"])["brand_id_count"].sum().reset_index()
UserClickGoodsCount.columns=["user_id","goods_count"]
print UserClickGoodsCount.head()
# 
train=pd.merge(train,UserClickGoodsCount,on="user_id",how="left")
test=pd.merge(test,UserClickGoodsCount,on="user_id",how="left")
# print test.head()
# print train.head()


# ---the rate of num of goods/num of brands 买一个商品（用类别来衡量）看的牌子种类多不多

train["goods/brands"]=np.rint(train["goods_count"]/train["brand_count"])
test["goods/brands"]=np.rint(test["goods_count"]/test["brand_count"])
# 
# print train.head()
# print test.head()

# there have an other one

# ---the feature about the cat, it shows whether a user like many category
UserClickCat=train.groupby(["user_id"])["cat_id"].value_counts()
UserClickCat=UserClickCat.rename("cat_count").reset_index()
print UserClickCat.head()
#   user_id  cat_id  cat_count
# 0        3   311.0        123
# 1        3  1056.0         46
# 2        3  1012.0         45
# 3        3   297.0         36
# 4        3   271.0         23

# ---the number of a user cat id in 3 month
# ---Don't calculate the mean sum(cat_count)/sum(cat_id)
UserClickCatCount=UserClickCat.groupby(["user_id"])["cat_id"].count().reset_index()
UserClickCatCount.columns=["user_id","cat_count"]
train=pd.merge(train,UserClickCatCount,on="user_id",how="left")
test=pd.merge(test,UserClickCatCount,on="user_id",how="left")

# ---the rate of the goodsnum/catsnum
train["goods/cats"]=np.rint(train["goods_count"]/train["cat_count"])
test["goods/cats"]=np.rint(test["goods_count"]/test["cat_count"])

# ---the feature combine the brand and the cat
# a cat he see how many brands  # the rate of a brand ,he cat how many goods, merge on brand 这个是根据商品建模了
train["brands/cat"]=np.rint(train["brand_count"]/train["cat_count"])
test["brands/cat"]=np.rint(test["brand_count"]/test["cat_count"])
print train.head()
print test.head()

   user_id    brand_id  brand_id_count
0        3  10004318.0              19
1        3  10005367.0              18
2        3  10013106.0              17
3        3  10020991.0              17
4        3  10000601.0              14


   user_id  brand_count
0        3          108
1        4           60
2       11           21
3       16           17
4       17          407


   user_id  goods_count
0        3          336
1        4          189
2       11           63
3       16           31
4       17         2435


   user_id  cat_id  cat_count
0        3   311.0        123
1        3  1056.0         46
2        3  1012.0         45
3        3   297.0         36
4        3   271.0         23


   user_id  spu_id  bought   date    brand_id  cat_id  click_count  \
0   320510  839213       0  03-10  10010631.0   271.0          141   
1   314210  369282       0  01-05  10001351.0   311.0          489   
2   381441  730484       0  01-05  10000866.0   311.0           39   
3   168858  902515       0  03-10  10028088.0   297.0          170   
4   270694  953270       0  01-30  10000223.0  1056.0          335   

   bought_count  bought_rate  brand_count  goods_count  goods/brands  \
0             0          0.0           56          141           3.0   
1             2          4.0          130          489           4.0   
2             0          0.0           17           39           2.0   
3             0          0.0           76          170           2.0   
4             4         12.0           79          335           4.0   

   cat_count  goods/cats  brands/cat  
0         10        14.0         6.0  
1          9        54.0        14.0  
2          8         5.0     

In [47]:
# train["bought_rate"]=((train["bought_count"]/train["click_count"])*100000000)/100000
# test["bought_rate"]=((test["bought_count"]/test["click_count"])*100000000)/100000


# train["bought_rate"]=train["bought_rate"]*100000%10000
# test["bought_rate"]=test["bought_rate"]*100000%10000

# train["bought_rate"]=np.rint(train["bought_rate"])
# test["bought_rate"]=np.rint(test["bought_rate"])
# 
# print train["bought_rate"].head()
# print test["bought_rate"].head()
print train.head()
print test.head()

   user_id  spu_id  bought   date    brand_id  cat_id  click_count  \
0   320510  839213       0  03-10  10010631.0   271.0          141   
1   314210  369282       0  01-05  10001351.0   311.0          489   
2   381441  730484       0  01-05  10000866.0   311.0           39   
3   168858  902515       0  03-10  10028088.0   297.0          170   
4   270694  953270       0  01-30  10000223.0  1056.0          335   

   bought_count  bought_rate  brand_count  goods_count  cat_count  goods/cats  \
0             0          0.0           56          141         10        14.0   
1             2          4.0          130          489          9        54.0   
2             0          0.0           17           39          8         5.0   
3             0          0.0           76          170          5        34.0   
4             4         12.0           79          335         17        20.0   

   brands/cat  goods/brands  
0         6.0           3.0  
1        14.0           4.0  
2 

In [10]:
# cat the na
missing_train = train.isnull().sum(axis=0).reset_index()# 这里的sum直接统计为True的值
missing_train.columns = ['column_name', 'missing_count']
print missing_train
# 
missing_test=test.isnull().sum(axis=0).reset_index()
missing_test.columns= ['column_name', 'missing_count']
print missing_test


# ---del the miss data

# train2=train.dropna()
# missing_train = train2.isnull().sum(axis=0).reset_index()# 这里的sum直接统计为True的值
# missing_train.columns = ['column_name', 'missing_count']
# print missing_train


# missing_df = missing_df.ix[missing_df['missing_count']>0]
# missing_df = missing_df.sort_values(by='missing_count')

#----cat the final data before model
# print train2.columns
# print test.columns
# Index([u'user_id', u'spu_id', u'bought', u'date', u'brand_id_x', u'cat_id',
#        u'click_count', u'bought_count', u'bought_rate', u'brand_id_count',
#        u'goods_count', u'goods/brands', u'cat_count', u'goods/cats',
#        u'brands/cat'],
#       dtype='object')
# Index([u'user_id', u'spu_id', u'prob', u'brand_id_x', u'cat_id',
#        u'click_count', u'bought_count', u'bought_rate', u'brand_id_count',
#        u'goods_count', u'goods/brands', u'cat_count', u'goods/cats',
#        u'brands/cat'],
#       dtype='object')
# del test["prob"]
# train2=train2.drop(["date"],axis=1)
# print train.columns
# print test.columns
# print len(train.columns)
# print len(test.columns)

     column_name  missing_count
0        user_id              0
1         spu_id              0
2         bought              0
3           date              0
4       brand_id              0
5         cat_id              0
6    click_count              0
7   bought_count              0
8    bought_rate              0
9    brand_count              0
10   goods_count              0
11  goods/brands              0
12     cat_count              0
13    goods/cats              0
14    brands/cat              0


     column_name  missing_count
0        user_id              0
1         spu_id              0
2           prob        5761091
3       brand_id              0
4         cat_id              0
5    click_count              0
6   bought_count              0
7    bought_rate              0
8    brand_count              0
9    goods_count              0
10  goods/brands              0
11     cat_count              0
12    goods/cats              0
13    brands/cat              0


In [11]:
# print train.shape
# print test.shape
# train=train.dropna()
# test=test.dropna()
# print train.shape
# print test.shape

# missing_train = train.isnull().sum(axis=0).reset_index()# 这里的sum直接统计为True的值
# missing_train.columns = ['column_name', 'missing_count']
# print missing_train
# 
# missing_test = test.isnull().sum(axis=0).reset_index()# 这里的sum直接统计为True的值
# missing_test.columns = ['column_name', 'missing_count']
# print missing_test

# train=train.dropna()

# missing_df = missing_df.ix[missing_df['missing_count']>0]
# missing_df = missing_df.sort_values(by='missing_count')

train.to_csv("data/train.csv")
test.to_csv("data/test.csv")

In [52]:
# train_sample=train
# test_sample=test
# train=train_sample.head()
# test=test_sample.head()
# print train
# print test
# train["bought"][3]=1
# print train["bought"]

# print train_sample.shape
# print test_sample.shape
# train=train_sample
# test=test_sample
# print train.shape
# print test.shape

# missing_train = train.isnull().sum(axis=0).reset_index()# 这里的sum直接统计为True的值
# missing_train.columns = ['column_name', 'missing_count']
# print missing_train
# 
# missing_test = test.isnull().sum(axis=0).reset_index()# 这里的sum直接统计为True的值
# missing_test.columns = ['column_name', 'missing_count']
# print missing_test

# train=train.dropna()
gc.collect()

0

In [3]:
train_all=pd.read_csv("data/train.csv")
test_all=pd.read_csv("data/test.csv")
del train_all["Unnamed: 0"]
del test_all["Unnamed: 0"]

columns_train=["user_id","bought","bought_count","bought_rate"]
columns_test=["user_id","bought_count","bought_rate"]

train=train_all[columns_train]
test=test_all[columns_test]



print train.head()
print test.head()

   user_id  bought  bought_count  bought_rate
0   320510       0             0          0.0
1   314210       0             2          4.0
2   381441       0             0          0.0
3   168858       0             0          0.0
4   270694       0             4         12.0
   user_id  bought_count  bought_rate
0    60750             4         20.0
1   595361             0          0.0
2    45427            21         17.0
3   443345             2          5.0
4   318510             5          5.0


In [6]:
y_train=train["bought"].values
enc=OneHotEncoder()
# feats=["user_id","spu_id","brand_id","cat_id","click_count","bought_count","bought_rate","brand_count",
#        "goods_count","goods/brands","cat_count","goods/cats","brands/cat"]
feats=["user_id","bought_count","bought_rate"]

print "11111111111"
for i,feat in enumerate(feats):
    print feat
    encd=enc.fit(np.array(list(train[feat])+list(test[feat])).reshape(-1,1))
    x_train=encd.transform(train[feat].values.reshape(-1,1))
    x_test=encd.transform(test[feat].values.reshape(-1,1))
    if i == 0:
        X_train, X_test = x_train, x_test
    else:
        X_train, X_test = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test))

11111111111
user_id


bought_count


KeyboardInterrupt: 

In [31]:
print X_train

  (3, 0)	1.0
  (4, 1)	1.0
  (1, 2)	1.0
  (0, 3)	1.0
  (2, 4)	1.0
  (1, 5)	1.0
  (2, 6)	1.0
  (0, 7)	1.0
  (3, 8)	1.0
  (4, 9)	1.0
  (4, 10)	1.0
  (2, 11)	1.0
  (1, 12)	1.0
  (0, 13)	1.0
  (3, 14)	1.0
  (0, 15)	1.0
  (3, 16)	1.0
  (1, 17)	1.0
  (2, 17)	1.0
  (4, 18)	1.0
  (2, 19)	1.0
  (0, 20)	1.0
  (3, 21)	1.0
  (4, 22)	1.0
  (1, 23)	1.0
  :	:
  (2, 35)	1.0
  (0, 36)	1.0
  (3, 37)	1.0
  (4, 38)	1.0
  (1, 39)	1.0
  (2, 40)	1.0
  (3, 40)	1.0
  (0, 41)	1.0
  (1, 42)	1.0
  (4, 42)	1.0
  (3, 43)	1.0
  (2, 44)	1.0
  (1, 45)	1.0
  (0, 46)	1.0
  (4, 47)	1.0
  (2, 48)	1.0
  (0, 49)	1.0
  (4, 50)	1.0
  (3, 51)	1.0
  (1, 52)	1.0
  (2, 53)	1.0
  (4, 54)	1.0
  (0, 55)	1.0
  (1, 56)	1.0
  (3, 57)	1.0


In [40]:
lr=LogisticRegression()

In [41]:
lr.fit(X_train,y_train)
print "22222222222"
proba_test=lr.predict_proba(X_test)[:,1]

22222222222


In [42]:
print proba_test

[ 0.37796232  0.25704719  0.3394084   0.39524905  0.42282906]


In [ ]:
proba_test.to_csv("data/result.csv")